In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('../data/raw/insurance.csv')

In [5]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,18,female,33.820,0,no,southeast,1630.66170
1,19,female,23.480,1,no,southeast,1836.80430
2,46,male,30.570,2,no,southeast,6632.35130
3,54,male,32.050,1,yes,southeast,31922.42950
4,21,male,21.345,4,no,northeast,1638.37255


In [6]:
df.shape

(50000, 7)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       50000 non-null  int64  
 1   sex       50000 non-null  object 
 2   bmi       50000 non-null  float64
 3   children  50000 non-null  int64  
 4   smoker    50000 non-null  object 
 5   region    50000 non-null  object 
 6   charges   50000 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 2.7+ MB


In [8]:
df.describe(include='all')

,age,sex,bmi,children,smoker,region,charges
count,50000.000000,50000,50000.000000,50000.000000,50000,50000,50000.000000
unique,NaN,2,NaN,NaN,2,4,NaN
top,NaN,male,NaN,NaN,no,southeast,NaN
freq,NaN,25176,NaN,NaN,38976,14197,NaN
mean,39.463120,NaN,30.713734,1.113760,NaN,NaN,13343.216363
std,14.117142,NaN,6.092727,1.212835,NaN,NaN,12131.222744
min,18.000000,NaN,17.291000,0.000000,NaN,NaN,1137.535900
25%,27.000000,NaN,26.600000,0.000000,NaN,NaN,4694.431800
50%,40.000000,NaN,30.300000,1.000000,NaN,NaN,9399.232775
75%,51.000000,NaN,34.570000,2.000000,NaN,NaN,17340.746925


In [9]:
df_cleaned = df.copy()

In [10]:
target = df_cleaned.pop('charges')

In [11]:
num_cols = list(df_cleaned.select_dtypes('number').columns)
cat_cols = list(set(df_cleaned.columns) - set(num_cols))

In [12]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [13]:
ohe = OneHotEncoder(sparse_output=False, drop='first')

In [14]:
features = ohe.fit_transform(df_cleaned[cat_cols])

In [15]:
features = pd.DataFrame(features, columns=ohe.get_feature_names_out())

In [17]:
scaler = StandardScaler()

In [18]:
features.head()

,region_northwest,region_southeast,region_southwest,smoker_yes,sex_male
0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,1.0


In [19]:
features[num_cols] = scaler.fit_transform(df_cleaned[num_cols])

In [20]:
from joblib import dump

In [21]:
dump(ohe, '../models/ohe.joblib')
dump(scaler, '../models/scaler.joblib')

['../models/scaler.joblib']

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_data, X_test, y_data, y_test = train_test_split(features, target, test_size=0.2, random_state=8)
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=8)

In [24]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(32000, 8)
(8000, 8)
(10000, 8)


In [25]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(32000,)
(8000,)
(10000,)


In [27]:
X_train.to_csv('../data/processed/X_train.csv', index = False)
X_val.to_csv('../data/processed/X_val.csv', index =False)
X_test.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_val.to_csv('../data/processed/y_val.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

In [30]:
pred_value = y_train.mean()

In [31]:
y_base = np.full((len(y_train), 1), pred_value)

In [33]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

In [34]:
print(mse(y_base, y_train, squared=False))
print(mae(y_base, y_train))

12116.584822448176
9118.852804794265
